# mcp_exchange_rates.py server with brave search 
# This is a demo do not use for personal trading decisions
## to get current fx and exchange rate trading recommendations
## prequisites
### 1. uses claude-sonnet-4
ANTHROPIC_API_KEY in .env file
### 2. use Brave Search API key
BRAVE_API in .env file




In [1]:
# import libraries
import os
import asyncio
from re import I
from agents import Agent, OpenAIChatCompletionsModel, Runner, trace
from agents.mcp import MCPServerStdio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display


In [2]:
load_dotenv(override=True)

# 1. Configure Claude client
claude_client = AsyncOpenAI(
    base_url="https://api.anthropic.com/v1",
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

# 2. Create Claude model
claude_model = OpenAIChatCompletionsModel(
    model="claude-sonnet-4-20250514",
    openai_client=claude_client
)

# 3. MCP server parameters (Brave Search)
search_params = {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
}
# 4. MCP server parameters (Forex)
exchange_rate_params = {
    "command": "uv", "args": ["run", "mcp_exchange_rates.py"]
}

In [3]:
print(exchange_rate_params)
print(search_params)

{'command': 'uv', 'args': ['run', 'mcp_exchange_rates.py']}
{'command': 'npx', 'args': ['-y', '@modelcontextprotocol/server-brave-search'], 'env': {'BRAVE_API_KEY': 'BSA3Y3e40UOjfP1M_2HMKvXmhf_NXRf'}}


In [6]:


try:
    #5. Connect to MCP server
    async with MCPServerStdio(
        params=exchange_rate_params, client_session_timeout_seconds=30) as forex_server:
        print("✅ Connected to Forex MCP server")
       # 6. Connect to MCP server
        async with MCPServerStdio(
            params=search_params, 
            client_session_timeout_seconds=60
        ) as search_server:
            
            print("✅ Connected to Brave Search MCP server")
            
            # 7. List available tools
            tools = await search_server.list_tools()
            forex_tools = await forex_server.list_tools()
            tools = tools + forex_tools
            print(f"�� Available tools: {tools}")
            # 8. instructions and request
            instructions="""You are a financial research assistant. You have access to web search tools and usd conversion tools.
                Use the search tools to find current information and provide accurate, up-to-date responses.
                Always cite your sources when providing information."""
            
            request ="you have USD 100000 to investment and want to make a profit. provide the latest exchange rate of \
                        usd/aud usd/eur, usd/nzd, usd/gbp, usd/can - print the rates in a table format\
                        print the inverse rates table of the above rates\
                        include the source and date of the information\
                        for each currency pair based get the latest news about the summary of the current market conditions for each currency pair. \
                        provide citations for source of news/information. \
                        and \
                        the current exchange rates recommend trades for usd/aud, usd/eur, usd/nzd, usd/gbp, usd/can \
                        provide a trade idea based on the news and the current market conditions"
            # 9. Create Claude agent with MCP server
            agent = Agent(
                name="claude_researcher",
                instructions=instructions,
                model=claude_model,
                mcp_servers=[search_server,forex_server]
            )
            
            print("✅ Claude agent created with MCP servers")
            
            # 10. Use the agent with tracing
            with trace("Claude and Forex Web Research"):
                result = await Runner.run(
                    agent,
                    request,
                    max_turns=25
                    )
            
            print("\n📊 Claude Agent Response:")
            print("=" * 50)
            # print(result.final_output)
            display(Markdown("This is a demo do not use for personal trading decisions"))
            display(Markdown(result.final_output))
            print("=" * 50)
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()



✅ Connected to Forex MCP server
✅ Connected to Brave Search MCP server
�� Available tools: [Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None), Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical l

This is a demo do not use for personal trading decisions

Based on the exchange rate data and market research, here's a comprehensive analysis for your $100,000 investment:

## Current Exchange Rates (as of September 6, 2025)

### Direct Rates (USD to Currency)

| Currency Pair | Exchange Rate | Source | Last Updated |
|---------------|---------------|---------|---------------|
| USD/AUD | 1.5262 | exchangerate-api.com | Sept 6, 2025 |
| USD/EUR | 0.8537 | exchangerate-api.com | Sept 6, 2025 |
| USD/NZD | 1.6980 | exchangerate-api.com | Sept 6, 2025 |
| USD/GBP | 0.7410 | exchangerate-api.com | Sept 6, 2025 |
| USD/CAD | 1.3827 | exchangerate-api.com | Sept 6, 2025 |

### Inverse Rates (Currency to USD)

| Currency Pair | Inverse Rate | Meaning |
|---------------|---------------|----------|
| AUD/USD | 0.6553 | 1 AUD = 0.6553 USD |
| EUR/USD | 1.1714 | 1 EUR = 1.1714 USD |
| NZD/USD | 0.5889 | 1 NZD = 0.5889 USD |
| GBP/USD | 1.3497 | 1 GBP = 1.3497 USD |
| CAD/USD | 0.7232 | 1 CAD = 0.7232 USD |

## Market Analysis Summary

### USD/AUD Market Conditions:
- **Current Rate**: 1.5262 (AUD/USD: 0.6553)
- **Market Outlook**: Bearish for AUD
- **Key Factors**: The AUD faces selling pressure with the USD stabilizing. Trading Economics shows AUD/USD rose to 0.6550 on September 5, up 0.42% from previous session¹
- **Risk Warning**: Bank of Sydney suggests risk of AUD heading below US$0.60 in first half 2025²

### USD/EUR Market Conditions:
- **Current Rate**: 0.8537 (EUR/USD: 1.1714)
- **Market Outlook**: Mixed to bearish for EUR
- **Key Factors**: J.P. Morgan expects EUR/USD to hit 1.19 by September 2025³, while FXStreet suggests a bearish technical outlook for 2025⁴
- **Support**: Euro supported by recent EU inflation data⁵

### Other Currency Analysis:
- **NZD**: Identified as worst G10 performer in August⁶
- **GBP**: Mid-table performance among G10 currencies
- **CAD**: Affected by broader USD strength trends

## Trading Recommendations for $100,000 Investment

### Recommended Trades:

1. **Short AUD/USD (25% allocation - $25,000)**
   - **Rationale**: Strong bearish outlook with risk of falling below 0.60
   - **Entry**: Current levels around 0.6553
   - **Target**: 0.60-0.62 range
   - **Stop Loss**: 0.68

2. **Long USD/CAD (20% allocation - $20,000)**
   - **Rationale**: USD strength trend likely to continue
   - **Entry**: Current 1.3827 level
   - **Target**: 1.42-1.45
   - **Stop Loss**: 1.35

3. **Short NZD/USD (20% allocation - $20,000)**
   - **Rationale**: Worst G10 performer, commodity currency weakness
   - **Entry**: Current 0.5889 level
   - **Target**: 0.55-0.57
   - **Stop Loss**: 0.62

4. **EUR/USD Long (15% allocation - $15,000)**
   - **Rationale**: J.P. Morgan's bullish forecast to 1.19
   - **Entry**: Current 1.1714 level
   - **Target**: 1.19-1.22
   - **Stop Loss**: 1.14

5. **Cash Reserve USD (20% allocation - $20,000)**
   - **Rationale**: Maintain liquidity for opportunities and risk management

### Risk Management Strategy:
- **Maximum risk per trade**: 2-3% of total capital
- **Diversification**: Spread across multiple currency pairs
- **Stop losses**: Strictly adhered to limit downside
- **Position sizing**: Conservative approach given market volatility

## Sources:
1. Trading Economics - Australian Dollar data
2. Bank of Sydney - AUD outlook 2025
3. J.P. Morgan Research - EUR/USD forecasts
4. FXStreet - EUR/USD technical analysis
5. RoboForex - EURUSD analysis
6. MUFG Research - Monthly FX Outlook

**Disclaimer**: This analysis is for informational purposes only. Forex trading involves substantial risk and may not be suitable for all investors. Past performance does not guarantee future results. Please consult with a qualified financial advisor before making investment decisions.